In [17]:
import sklearn
import pandas as pd
import matplotlib as plt
import seaborn

In [7]:
df = pd.read_csv("covid19-global-forecast/train.csv")
df

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,33.0000,65.0000,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,33.0000,65.0000,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,33.0000,65.0000,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,33.0000,65.0000,2020-01-26,0.0,0.0
...,...,...,...,...,...,...,...,...
16751,26374,NaN,Zambia,-15.4167,28.2833,2020-03-16,0.0,0.0
16752,26375,NaN,Zambia,-15.4167,28.2833,2020-03-17,0.0,0.0
16753,26376,NaN,Zambia,-15.4167,28.2833,2020-03-18,2.0,0.0
16754,26377,NaN,Zambia,-15.4167,28.2833,2020-03-19,2.0,0.0


In [10]:
df.describe()

,Id,Lat,Long,ConfirmedCases,Fatalities
count,16756.000000,16697.000000,16697.000000,16756.000000,16756.000000
mean,13189.500000,26.337324,5.039603,271.750119,9.221533
std,7624.686932,22.963063,79.935267,3261.152305,141.524582
min,1.000000,-41.454500,-157.498300,0.000000,0.000000
25%,6595.250000,13.193900,-71.530100,0.000000,0.000000
50%,13189.500000,33.000000,10.000000,0.000000,0.000000
75%,19783.750000,42.506300,65.000000,7.000000,0.000000
max,26378.000000,72.000000,174.886000,67800.000000,4032.000000


In [13]:
df["Country/Region"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cruise Ship', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'Equatorial Guinea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Finland', 'France', 'French Guiana', 'Gabon', 'Gambia, The',
       'Georgia', 'Germany', 'Ghana', 'Greece', 'Greenland', 'Guadeloupe',
       'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guyana', 'Holy See',
       'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia

In [16]:
data = df[df["Country/Region"].isin(["China", "US", "Italy", "Spain"])]
data

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
2773,4372,Anhui,China,31.8257,117.2264,2020-01-22,1.0,0.0
2774,4373,Anhui,China,31.8257,117.2264,2020-01-23,9.0,0.0
2775,4374,Anhui,China,31.8257,117.2264,2020-01-24,15.0,0.0
2776,4375,Anhui,China,31.8257,117.2264,2020-01-25,39.0,0.0
2777,4376,Anhui,China,31.8257,117.2264,2020-01-26,60.0,0.0
...,...,...,...,...,...,...,...,...
16043,25258,Wyoming,US,42.7560,-107.3025,2020-03-16,3.0,0.0
16044,25259,Wyoming,US,42.7560,-107.3025,2020-03-17,11.0,0.0
16045,25260,Wyoming,US,42.7560,-107.3025,2020-03-18,15.0,0.0
16046,25261,Wyoming,US,42.7560,-107.3025,2020-03-19,18.0,0.0
